# Непересекающиеся множества

Методы:
    
    MakeSet(x)  создать одноэлементное множество {x}
    Find(x)     выдать ID множества, содержащего x
    Union(x, y) объединение двух множеств

## Простейшие реализации

Наивная реализация на **массиве**: элементы - числа, массив 932475618, множества {9, 3, 2, 4, 7} {5} {6, 1, 8}, ID - минимальный элемент множества

    MakeSet(i)                          # O(1)
    smallest[i] = i

    Find(i)                             # O(1)
    return smallest[i]                  

    Union(i, j)                         # O(n)
    i_id = Find(i), j_id = Find(j)
    if i_id == j_id
        return                          # уже объеденины
    m = min(i_id, j_id)
    for k from 1 to n:
        if smallest[k] in {i_id, j_id}
            smallest[k] = m

Наивная реализаци корнем в 5.

## Представление множеств в виде деревьев

И тут на выручну спешит массив родителей, с которым развлекались ранее. 

    parent(i):  6 4 4 4 5 6 9 6 4
            i:  1 2 3 4 5 6 7 8 9

{5}, {6, 1, 8} и {9, 3, 2, 4, 7} в виде деревьев (5), (6 (1,8)) и (4 (2,3,9(7)))

    MakeSet(i):         # O(1)
    parent[i] = i
    rank[i] = 0         # см. далее

    Find(i)             # O(height)
    while i != parent[i]:
        i = parent[i]
    return i

## Объединение по рангу

С объединением другая история. (6 (1,8)) + (4 (2,3,9 (7))) = (6 (1,8 (4 (2,3,9 (7))))) или (4 (2,3,(6 (1,8)),9 (7))). Второй вариант лучше, т.к. высота меньше

    Union(i, j)                         # O(n)
    i_id = Find(i), j_id = Find(j)
    if i_id == j_id
        return                          # уже в одном дереве
    if rank[i_id] > rank[j_id]
        parent[j_id] = i_id
    else
        parent[i_id] = j_id
        if rank[i_id] == rank[j_id]     # единственный случай когда высота объединенного дерева увеличится, когда их высоты равны
            rank[j_id] += 1             # и одно окажается на 1 уровень ниже

    parent(i):  3 2 2 2 2 2
            i:  1 2 3 4 5 6
         rank:  0 2 1 0 0 0
    (2 (5,4,3 (1), 6)))
        
Лемма: rank\[i] это всегда высота поддерева с корнем в i, и высота дерева в лесе всегда <= log2(n)

## Сжатие путей

Просто добавить рекурсию

    Find(i)                             # O(height)
    while i != parent[i]:
        parent[i] = Find(parent[i])     # рекурсивно "переподвесить" всех потомков к корню перед объединением
    return parent[i]

## Анализ времени работы

Доказывается через итерированный логарифм log\*n (сколько раз применить log2 к n чтобы перевести его к <= 1). Сжимает она очень быстро, на практике значение больше 5 не используется (log\*n = 5 значит n=2^(2^(2^(2^2)))) = 2^65536), т.е. на реальных n сложность O(log\*n) = O(<=5).

При сжатии путей rank\[i] уже не будет высотой поддерева.


## Задача. Объединение таблиц

Цель - реализовать симуляцию объединения таблиц в базе данных.   
В базе данных есть n таблиц, пронумерованных от 1 до n, над одним и тем же множеством столбцов (атрибутов). Каждая таблица содержит либо реальные записи в таблице, либо символьную ссылку на другую таблицу. Изначально все таблицы содержат реальные записи, и i-я таблица содержит r i записей. Ваша цель — обработать m запросов типа (destination i , source i ):  

    1. Рассмотрим таблицу с номером destination i . Пройдясь по цепочке символьных ссылок, найдём номер реальной таблицы, на которую ссылается эта таблица:
        пока таблица destination i содержит символическую ссылку:
            destination i ← symlink(destination i )
    2. Сделаем то же самое с таблицей source i .
    3. Теперь таблицы destination i и source i содержат реальные записи. Если destination i 6 = source i , скопируем все записи из таблицы source i в таблицу destination i , очистим таблицу source i и пропишем в неё символическую ссылку на таблицу destination i .
    4. Выведем максимальный размер среди всех n таблиц. Размером таблицы называется число строк в ней. Если таблица содержит символическую ссылку, считаем её размер равным нулю.

Формат входа. Первая строка содержит числа n и m — число таблиц и число запросов, соответственно. Вторая строка содержит n целых чисел r 1 , . . . , r n — размеры таблиц. Каждая из последующих m строк содержит два номера таблиц destination i и source i , которые необходимо объединить.  
Формат выхода. Для каждого из m запросов выведите максимальный размер таблицы после соответствующего объединения.  
Ограничения. 1 ≤ n, m ≤ 100 000; 0 ≤ r i ≤ 10 000; 1 ≤ destination i , source i ≤ n.


In [41]:
SAMPLE = "5 5\n1 1 1 1 1\n3 5\n2 4\n1 4\n5 4\n5 3", "6 4\n10 0 5 0 3 3\n6 6\n6 5\n5 4\n4 3"
OUTPUT = ...
READER = (x for x in SAMPLE[0].split('\n')); input = lambda: next(READER)

def find(i, parent):
    while i != parent[i]:
        parent[i] = parent[parent[i]]
        i = parent[i]
    return i

def union(d, s, parent, rank):
    global maximum
    dest = find(d, parent)
    source = find(s, parent)
    if dest != source:
        parent[source] = dest
        rank[dest] += rank[source]
        rank[source] = 0
        if rank[dest] > maximum:
            maximum = rank[dest]

n, m = list(map(int, input().split()))

links = list(range(n))
tables = list(map(int, input().split()))
maximum = max(tables)

for _ in range(m):
    dest, source = list(map(int, input().split()))
    union(dest - 1, source - 1, links, tables)
    print(maximum)


2
2
3
5
5


## Автоматический анализ программ

При автоматическом анализе программ возникает такая задача на систему равенств и неравенств:  

Проверить, можно ли присвоить переменным целые значения, чтобы выполнить заданные равенства вида x i = x j и неравенства вида x p != x q.  
Вход. Число переменных n, а также список равенств вида x i = x j и неравенства вида x p != x q .  
Выход. Проверить, выполнима ли данная система.


Формат входа. Первая строка содержит числа n, e, d. Каждая из следующих e строк содержит два числа i и j и задаёт равенство x i = x j . Каждая из следующих d строк содержит два числа i и j и задаёт неравенство x i != x j . Переменные индексируются с 1: x 1 , . . . , x n.  
Формат выхода. Выведите 1, если переменным x 1 , . . . , x n можно присвоить целые значения, чтобы все равенства и неравенства выполнились. В противном случае выведите 0.  
Ограничения. 1 ≤ n ≤ 10 5 ; 0 ≤ e, d; e + d ≤ 2 · 10 5 ; 1 ≤ i, j ≤ n.  

In [74]:
SAMPLE = "4 6 0\n1 2\n1 3\n1 4\n2 3\n2 4\n3 4", "4 6 1\n1 2\n1 3\n1 4\n2 3\n2 4\n3 4\n1 2", "4 0 6\n1 2\n1 3\n1 4\n2 3\n2 4\n3 4"
OUTPUT = 1, 0, 1
READER = (x for x in SAMPLE[1].split('\n')); input = lambda: next(READER)

# т.е. задаются f(e) множеств равных элементов, проверить что ни одна пара из d неравных элементов не находятся в одном множестве
# -> создаем множества равных, тогда у неравных должны быть разные родители

n, e, d = list(map(int, input().split()))

def test(size, eq, neq):
    
    def find(i):
        while i != vs[i]:
            vs[i] = vs[vs[i]]
            i = vs[i]
        return i

    vs = list(range(size))

    # make sets
    for _ in range(eq):
        i, j = list(map(int, input().split()))
        vs[find(i - 1)] = find(j - 1)             # вешаем на любого, т.к. ранг не важен

    # test sets
    for _ in range(neq):
        i, j = list(map(int, input().split()))
        if find(i - 1) == find(j - 1):
            return 0
    
    return 1

print(test(n, e, d))

0
